# Assignment 2 - Recurrent Neural Networks



## Programming (Full points: 100)

In this assignment, our goal is to use PyTorch to implement Recurrent Neural Networks (RNN) for sentiment analysis task. Sentiment analysis is to classify sentences (input) into certain sentiments (output labels), which includes positive, negative and neutral.

We will use a benckmark dataset, SST, for this assignment.
* we download the SST dataset from torchtext package, and do some preprocessing to build vocabulary and split the dataset into training/validation/test sets. You don't need to modify the code in this step.


In [26]:
import copy
import torch
from torch import nn
from torch import optim
import torchtext
from torchtext import data
from torchtext import datasets

TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.LabelField()

# load data splits
train_data, val_data, test_data = datasets.SST.splits(TEXT, LABEL)

# build dictionary
TEXT.build_vocab(train_data)
LABEL.build_vocab(train_data)

#print(train_data)

# hyperparameters
vocab_size = len(TEXT.vocab)
label_size = len(LABEL.vocab)
padding_idx = TEXT.vocab.stoi['<pad>']
embedding_dim = 128
hidden_dim = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# build iterators
train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), 
    batch_size=32)
   


In [27]:
#Printing the sample values in training data
print(train_data[0].__dict__.keys())
print(train_data[0].__dict__.values())
print(LABEL.vocab.freqs.most_common(5))

dict_keys(['text', 'label'])
dict_values([['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'century', "'s", 'new', '``', 'conan', "''", 'and', 'that', 'he', "'s", 'going', 'to', 'make', 'a', 'splash', 'even', 'greater', 'than', 'arnold', 'schwarzenegger', ',', 'jean-claud', 'van', 'damme', 'or', 'steven', 'segal', '.'], 'positive'])
[('positive', 3610), ('negative', 3310), ('neutral', 1624)]


In [5]:
#Printing number of records in training/Validation/testing 
print(f"No of records in training data: {len(train_data)}")
print(f"No of records in test data: {len(test_data)}")
print(f"No of records in validation data: {len(val_data)}")

print(f"No of records in training iteration: {len(train_iter)}")
print(f"No of records in test iteration: {len(test_iter)}")
print(f"No of records in validation iteration: {len(val_iter)}")

print(f"Vocabulary size : {vocab_size}")
print(f"Number of classes : {label_size}")



No of records in training data: 8544
No of records in test data: 2210
No of records in validation data: 1101
No of records in training iteration: 267
No of records in test iteration: 70
No of records in validation iteration: 35
Vocabulary size : 16583
Number of classes : 3


In [22]:
#Defining the device to use
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

* define the training and evaluation function in the cell below.
### (25 points)


In [9]:
# Training model
def train_model(model, iterator):
     model.train()
     total_loss=[]
     total_accuracy=[]
     for i in range(epoch):
        epoch_loss = 0
        epoch_accuracy=0  
        loss_int=0
        accuracy_int=0
         
        for batch in iterator:            
            text = batch.text
            label=batch.label
            optimizer.zero_grad()            
            out = model(text)        #Forward Pass   
            loss = criterion(out, label)            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() #Loss Calculation

             # Calculate accuracy
            predictions = out.argmax(1)
            correct = (predictions == label).sum().item()
            accuracy = correct / len(label)
            epoch_accuracy+=accuracy

        loss_int="{:.4f}".format(epoch_loss / len(iterator))
        accuracy_int="{:.4f}".format(epoch_accuracy / len(iterator))
        total_loss.append(loss_int)
        total_accuracy.append(accuracy_int)
        print(f'Epoch {i+1}/{epoch}, Loss: {loss_int}, Accuracy:{accuracy_int}')

     return total_loss,total_accuracy 

#Evaluate the model
def evaluate_model(model, iterator):
    model.eval()
    epoch_loss = 0
    epoch_accuracy = 0
    
    with torch.no_grad():
        for batch in iterator:
            text = batch.text
            label = batch.label
            
            # Forward pass
            out = model(text)
            
            # Calculate loss and accuracy
            loss = criterion(out, label)
            
            # Calculate accuracy
            predictions = out.argmax(1)
            correct = (predictions == label).sum().item()
            accuracy = correct / len(label)
            
            epoch_loss += loss.item()
            epoch_accuracy += accuracy
            
    final_loss="{:.4f}".format(epoch_loss / len(iterator))
    final_accuracy="{:.4f}".format(epoch_accuracy / len(iterator))
    return final_loss, final_accuracy

* build a RNN model for sentiment analysis in the cell below.
We have provided several hyperparameters we needed for building the model, including vocabulary size (vocab_size), the word embedding dimension (embedding_dim), the hidden layer dimension (hidden_dim), the number of layers (num_layers) and the number of sentence labels (label_size). Please fill in the missing codes, and implement a RNN model.
### (40 points)

In [30]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, label_size, padding_idx):
        super(RNNClassifier, self).__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.num_layers = 1
        
        # add the layers required for sentiment analysis.
        # Embedding Layer
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim, padding_idx=padding_idx) 
        #RNN Layer
        self.rnn=nn.RNN(embedding_dim,hidden_dim, num_layers=self.num_layers,batch_first=True)
        #output Layer
        self.output = nn.Linear(hidden_dim, label_size)
        # Activation function for classification
        self.activation = nn.Sigmoid() if label_size == 1 else nn.LogSoftmax(dim=1)

    def zero_state(self, batch_size): 
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)  #hidden state
        cell = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)  # cell state
        return hidden,cell

    def forward(self, text):
        #implement the forward function of the model.
        embedding = self.embedding(text)

        #Initializing hidden state 
        batch_size=text.size(0)
        #print(batch_size)
        hidden,cell=self.zero_state(batch_size)

        #RNN
        out,hidden=self.rnn(embedding,hidden)

        #output layer
        hidden=hidden[-1]
        out=self.activation(self.output(hidden))
        
        return out

* train the model and compute the accuracy in the cell below.
### (20 points)

In [18]:
#Initializing paramters
learning_rate=0.0001
epoch=5
batch_size=32
output_dim=3

#Model Defination
model=RNNClassifier(vocab_size, embedding_dim, hidden_dim, output_dim, padding_idx).to(device)
model=model.to(device)

#Optimizer and Loss function
criterion=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
     
#Training Model with Training Iteration
print("Training the model with training Iteration")
training_loss, training_accuracy=train_model(model,train_iter)

#Training Model with Validation Iteration
print("Training the model with Validation Iteration")
valid_loss,valid_accuracy=train_model(model,val_iter)

#Evaluating the model with testing Iteration
print("Evaluating the model with testing Iteration")
test_loss, test_accuracy = evaluate_model(model, test_iter)
print(f"Test Loss: {test_loss}, Test Accuracy: {float(test_accuracy)*100}%")

Training the model with training Iteration
Epoch 1/5, Loss: 1.0547, Accuracy:0.4222
Epoch 2/5, Loss: 1.0473, Accuracy:0.4225
Epoch 3/5, Loss: 1.0464, Accuracy:0.4243
Epoch 4/5, Loss: 1.0455, Accuracy:0.4217
Epoch 5/5, Loss: 1.0452, Accuracy:0.4239
Training the model with Validation Iteration
Epoch 1/5, Loss: 1.0146, Accuracy:0.5038
Epoch 2/5, Loss: 1.0003, Accuracy:0.5400
Epoch 3/5, Loss: 0.9706, Accuracy:0.5525
Epoch 4/5, Loss: 0.9543, Accuracy:0.5609
Epoch 5/5, Loss: 0.9372, Accuracy:0.5707
Evaluating the model with testing Iteration
Test Loss: 1.0615, Test Accuracy: 52.32%


* try to train a model with better accuracy in the cell below. For example, you can use different optimizers such as SGD and Adam. You can also compare different hyperparameters and model size.
### (15 points), to obtain FULL point in this problem, the accuracy needs to be higher than 70%
**Answer:**<br>
There is a new file containing the code which tries to improve the accuracy of the model than the current one.
Created new file for better understanding.<br>
File Name - Assign2_LSTM_model